In [1]:
# from huggingface_hub import hf_hub_download 
import torch
from datasets import load_dataset, load_from_disk

import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, TFBertForMaskedLM, BertForMaskedLM
import tensorflow as tf
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torch import nn
from huggingface_hub import notebook_login
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map, dispatch_model

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, NMF, non_negative_factorization
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import re
import os
import glob
import itertools

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string

2024-08-09 17:38:05.646132: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 17:38:05.646302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 17:38:05.810297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
print(transformers.__version__)

4.42.3


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

2.1.2+cpu
False


In [4]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
model = BertForMaskedLM.from_pretrained("google-bert/bert-base-cased")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
dataset = load_dataset("yelp_polarity")

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [7]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

# Inference

Learnings:
1. The outputs of MLM BERT are logits corresponding to each word, to get prediction of the masked word, look at the logits corresponding to \[MASK\] token

In [40]:
# Without fine-tuning
inputs = tokenizer("I'm working at TD as a [MASK] Planner, I'm reporting [MASK] Hannah [MASK]. She is a piece of [MASK], I'm extremely [MASK] by her. ", return_tensors="pt")

In [60]:
inputs = tokenizer("I'm working at TD as a [MASK] Planner.", return_tensors="pt")

In [61]:
inputs

{'input_ids': tensor([[  101,   146,   112,   182,  1684,  1120, 15439,  1112,   170,   103,
          7382,  2511,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [62]:
with torch.no_grad():
    logits = model(**inputs).logits

In [63]:
logits.size()

torch.Size([1, 14, 28996])

In [64]:
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

In [65]:
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

In [66]:
all_token_id = logits[0].argmax(axis=-1)

In [67]:
tokenizer.decode(predicted_token_id)

'Senior'

In [68]:
all_token_id

tensor([  119,   146,   112,   182,  1684,  1120, 15439,  1112,   170,  4308,
         7382,  2511,   119,   119])

In [69]:
for i in range(len(all_token_id)):
    print("Input token: {}, generated token ID: {}, generated token: {}".format(
          tokenizer.decode(inputs["input_ids"][0][i]), 
          all_token_id[i], 
          tokenizer.decode(all_token_id[i]))
         )

Input token: [CLS], generated token ID: 119, generated token: .
Input token: I, generated token ID: 146, generated token: I
Input token: ', generated token ID: 112, generated token: '
Input token: m, generated token ID: 182, generated token: m
Input token: working, generated token ID: 1684, generated token: working
Input token: at, generated token ID: 1120, generated token: at
Input token: TD, generated token ID: 15439, generated token: TD
Input token: as, generated token ID: 1112, generated token: as
Input token: a, generated token ID: 170, generated token: a
Input token: [MASK], generated token ID: 4308, generated token: Senior
Input token: Plan, generated token ID: 7382, generated token: Plan
Input token: ##ner, generated token ID: 2511, generated token: ##ner
Input token: ., generated token ID: 119, generated token: .
Input token: [SEP], generated token ID: 119, generated token: .


In [45]:
tokenizer.decode([  101,   146,   112,   182,  1684,  1120, 15439,  1112,   170,   103,
          7382,  2511,   102])

"[CLS] I'm working at TD as a [MASK] Planner [SEP]"